In [1]:
import sys
import os
import io

# Add the parent directory to the sys.path
sys.path.append(os.path.join(os.getcwd(), '..'))

In [2]:
import glob
import csv

import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import kurtosis, skew, rankdata
from scipy.signal import argrelextrema
from scipy.stats.kde import gaussian_kde

from rcv_learning.rcv_distribution import *
from rcv_learning.rcv_dimensionality import *

/var/folders/g2/25z774tn0599smwks8_v_k200000gp/T/ipykernel_88190/4019494009.py:10: DeprecationWarning: Please use `gaussian_kde` from the `scipy.stats` namespace, the `scipy.stats.kde` namespace is deprecated.
  from scipy.stats.kde import gaussian_kde


In [3]:
def compute_ecdf(data):
    """Compute the empirical cumulative distribution function (ECDF) of data."""
    sorted_data = np.sort(data)
    n = len(data)
    return sorted_data, np.arange(1, n+1) / n

def dip_statistic(data):
    """Compute Hartigan's dip statistic for data."""
    data = np.sort(data)
    n = len(data)
    
    # Compute the empirical CDF
    x, ecdf = compute_ecdf(data)
    
    # Compute the greatest difference on the left and right side for each data point
    U = np.arange(1, n+1) / n
    D = U - np.arange(0, n) / n
    
    # Calculate the difference between the empirical distribution function and the 
    # unimodal distribution function that minimizes that maximum difference.
    low_dip = (ecdf - D).clip(min=0)
    up_dip = (U - ecdf).clip(min=0)
    both_dips = np.column_stack((low_dip, up_dip))
    
    # The dip statistic is the maximum difference over all data points
    dip = np.max(both_dips)
    
    return dip

In [6]:
# Search for CSV files in the directory and its subdirectories
csv_files = glob.glob("../rcv_elections_database/**/*.csv", recursive=True)

# Determine save behavior
save = True

# Iterate over the CSV files
for file_path in csv_files:

    # Get the filename only
    filename = file_path.split('/')[-1]  
    print(file_path)

    try:
        csv = file_path
        filename_prefix = "plots/" + filename

        # Check if the plot already exists
        if os.path.isfile(f"{filename_prefix}_hist.png"):
            print("Skipping hist plot: {}".format(file_path))
            continue
            
        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)
        print("Normalized distances:", normalized_distances)
        plot_rcv_analysis(mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, save=save, filename=filename_prefix)

        # Get the consistency points for the bimodality analysis
        points = get_consistency_points(csv)
        print("Consistency points:", points)

        # Create a list of data points
        data_points = []
        for key, value in points.items():
            data_points.extend([key] * value)

        # Convert to numpy array
        data_points = np.array(data_points)

        # Calculate skewness and kurtosis
        g = skew(data_points)
        k = kurtosis(data_points)

        # Calculate KDE without plotting
        density = gaussian_kde(data_points)
        x_vals = np.linspace(min(data_points), max(data_points), 1000)
        y_vals = density(x_vals)

        # Identify local maxima in the KDE
        maxima_indices = argrelextrema(y_vals, np.greater)

        # Get the x-values of the maxima
        modes = x_vals[maxima_indices]

        # Check if there are at least two modes
        if len(modes) >= 2:
            mode1, mode2 = modes[:2]

            # Calculate the amplitudes of the two modes
            amp1 = density(mode1)[0]
            amp2 = density(mode2)[0]

            # Calculate the amplitude ratio with higher amplitude as denominator
            if amp1 > amp2:
                amplitude_ratio = amp2 / amp1
            else:
                amplitude_ratio = amp1 / amp2

            # Calculate x-axis distance between the two modes
            mode_distance = abs(mode2 - mode1)

        else:
            mode1, mode2 = None, None
            amplitude_ratio = None
            mode_distance = None

        try:
            # Split the data into two groups based on proximity to the modes
            data_group1 = [point for point in data_points if abs(point - mode1) < abs(point - mode2)]
            data_group2 = [point for point in data_points if abs(point - mode1) > abs(point - mode2)]

            # Display mean and variance for each group
            mu1, sigma1_sq = np.mean(data_group1), np.var(data_group1)
            mu2, sigma2_sq = np.mean(data_group2), np.var(data_group2)

            # Calculate Ashman's D statistic (D > 2)
            ashmans_D = abs(mu1 - mu2) / np.sqrt((sigma1_sq + sigma2_sq) / 2)
        except:
            mode_distance = 404
            amplitude_ratio = 404
            ashmans_D = 404

        # Calculate Sarle's bimodality coefficient b (b > 5/9)
        n = len(data_points)
        sarle = (g**2 + 1) / (k + 3 * (n-1)**2 / ((n-2) * (n-3)))

        # Calculate Hartigan's dip statistic (calculate p value)
        hartigan_dip = dip_statistic(data_points)

        # Prepare data for histogram
        data_list = [x for x, count in points.items() for _ in range(count)]
        normalized_points = []
        normalized_names = []
        for name in normalized_distances:
            normalized_names.append(name)
            normalized_points.append(normalized_distances[name])

        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(data_list, bins=50, density=True, alpha=0.7)
        plt.title('Histogram of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display measures in a text box
        text_str = f"Skewness: {g:.2f}\nKurtosis: {k:.2f}\nMode Distance: {mode_distance:.2f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        if save:
            plt.savefig(f"{filename_prefix}_hist.png", bbox_inches='tight')
            plt.close()
        else:
            plt.show()

        # Plot kernal density estimation
        plt.figure(figsize=(10, 6))
        sns.kdeplot(data_list, fill=True)
        plt.title('Kernel Density Estimation of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display bimodality measures in a text box
        text_str = f"Ashman's D: {ashmans_D:.10f}\nSarle's Coefficient: {sarle:.10f}\nHartigan's Dip: {hartigan_dip:.10f}\nAmplitude Ratio: {amplitude_ratio:.10f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        if save:
            plt.savefig(f"{filename_prefix}_kde.png", bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    except Exception as e:
        print("Error in file: {}".format(file_path))
        print(e)

../rcv_elections_database/CandidateDetails.csv
Error in file: ../rcv_elections_database/CandidateDetails.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/SingleWinnerRCV.csv
Error in file: ../rcv_elections_database/SingleWinnerRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/MatchedElections.csv
Error in file: ../rcv_elections_database/MatchedElections.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/SequentialRCV.csv
Error in file: ../rcv_elections_database/SequentialRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/ProportionalRCV.csv
Error in file: ../rcv_elections_database/ProportionalRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/proportional/Minneapolis_11072017_BoardofEstimateandTaxation.csv
Skipping hist plot: ../rcv_el

/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:177: RuntimeWarning: invalid value encountered in divide
  frequencies = counts / mentioned_together
/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'D': 0.0, 'C': 0.9642454182431139, 'B': 2.1258304988909043, 'A': 3.0}


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {2.1389094004470097: 1, 1.2613456724844778: 1, 0.4408950834610708: 1}
Error in file: ../rcv_elections_database/classic/custom.csv
division by zero
../rcv_elections_database/sequential/Vineyard_11052019_CityCouncil_tab2.csv
Error in file: ../rcv_elections_database/sequential/Vineyard_11052019_CityCouncil_tab2.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/sequential/Payson_11052019_CityCouncil_tab1.csv
Error in file: ../rcv_elections_database/sequential/Payson_11052019_CityCouncil_tab1.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/sequential/WoodlandHills_11022021_CityCouncil.csv
Skipping hist plot: ../rcv_elections_database/sequential/WoodlandHills_11022021_CityCouncil.csv
../rcv_elections_database/sequential/Vineyard_11052019_CityCouncil_tab1.csv
Error in file: ../rcv_elections_database/sequential/Vineyard_11052019_CityCouncil_tab1.csv
zero-size array to reducti

...

Consistency points: {1.0: 3475, 0.0: 2213, 0.25: 1289}
../rcv_elections_database/single/NewYorkCity_06222021_DEMCouncilMember7thCouncilDistrict.csv
Skipping hist plot: ../rcv_elections_database/single/NewYorkCity_06222021_DEMCouncilMember7thCouncilDistrict.csv
../rcv_elections_database/single/Vineyard_11022021_Mayor.csv
Skipping hist plot: ../rcv_elections_database/single/Vineyard_11022021_Mayor.csv
../rcv_elections_database/single/Albany_11082022_BoardofEducation.csv
Skipping hist plot: ../rcv_elections_database/single/Albany_11082022_BoardofEducation.csv
../rcv_elections_database/single/SanLeandro_11032020_MemberCityCouncilDist2SanLeandroRCV.csv


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'BRYAN AZEVEDO': 0.0, 'ED HERNANDEZ': 1.0}


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {1.0: 5560, 0.0: 6439, 0.25: 12103}
